# Configuration

In [ ]:
# Pre-requisite Configuration
# ------------------------------------------------------------------------------ #
# For testing
# ------------------------------------------------------------------------------ #

%load_ext autoreload
%autoreload 2

import os 
from modules.util.config import get_config_by_id, get_config_global
from modules.util.helpers import Logger
from modules.util.database import SQLAlchemyClient


# ------------------------------------------------------------------------------ #
# Configuration
# ------------------------------------------------------------------------------ #

CONFIG_ID = 'dca-test'
TRANSFORM = get_config_by_id(CONFIG_ID)["transform"]["indicator"]
EXTRACTION_DIR = TRANSFORM["directory"]
REPORTS_DIR = f"{EXTRACTION_DIR}/reports"
CONFIG_GLOBAL = get_config_global().get('indicators').get('transform')

# ------------------------------------------------------------------------------ #
# Create directories
# ------------------------------------------------------------------------------ #
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)

# ------------------------------------------------------------------------------ #
# Logger
# ------------------------------------------------------------------------------ #

log = Logger.get_logger(CONFIG_ID)

Logger.blank_line(log)
log.info("** TRANSFORM - INDICATORS **")
Logger.blank_line(log)

log.info(f"Extraction Directory: {EXTRACTION_DIR}")
log.info(f"Reports Directory: {REPORTS_DIR}")

db = SQLAlchemyClient(CONFIG_ID)

# UDR - Generation

In this step a UDR (User Decision Report) is generated. The UDR contains a list of necessary fields from the source system and proposes a few fields which need user confirmation before migrating the data to the target APM system. Following are the proposals.

| Source |  Target Proposal | Conditions |
| ------ | ---------------- | ---------- |
| Indicator Groups | Indicator Positions | Special characters are stripped off / First 20 characters of the indicator group
| Indicators (datatypes) | ERP - Characteristics | Proposal based on the datatype, scale & precision of an indicator to be created as characteristic in ERP
| **configuration** | Indicator Category | A static value configured in the configuration is proposed as Indicator Category for APM

We have a list of views to be created in the database before creating a UDR.

| View | Purpose |
| ---- | ------ |
| `V_TRANSFORM_EQU_INDICATORS` | Transforms & consolidates data for `EQU` technical objects
| `V_TRANSFORM_FLOC_INDICATORS` | Transforms & consolidates data for `FLOC` technical objects
| `V_TRANSFORM_INDICATORS` | Consolidates (union) of EQU & FLOC indicators


In [ ]:
# UDR: Generation
# ------------------------------------------------------------------------------ #
#standard imports
import pandas as pd

# custom imports
from modules.util.helpers import convert_dataframe
from modules.util.database import V_Transform_Indicators

# constants
udr_file = f"{REPORTS_DIR}/UDR_APM_Indicators.csv"
defaults = TRANSFORM.get("defaults")
map_indicator_category = defaults.get("apm_default_indicator_category")
map_indicator_position = defaults.get("apm_default_indicator_position")
erp_mapping = CONFIG_GLOBAL.get("characteristic").get("erp_mapping")

Logger.blank_line(log)
log.info("Transform: Generate UDR")
Logger.blank_line(log)

def propose_erp_characteristic(row):    
    ind_datatype = row['indicators_dataType']
    ind_scale = row['indicators_scale']
    ind_precision = row['indicators_precision']

    prefix = map_indicator_position.get('prefix')
    sep = map_indicator_position.get('seperator')
    map_value = map_indicator_position.get(ind_datatype)


    if (ind_scale != '0' or ind_precision != '0') and (ind_datatype != None):
        return f"{f"{prefix}{sep}" if prefix else ''}{map_value}{sep}{ind_scale}{sep}{ind_precision}"
    else:
        return f"{f"{prefix}{sep}" if prefix else ''}{map_value}"

def propose_apm_indicator_position(row):
    internalId = row['indicatorGroups_internalId']
    description = row['indicatorGroups_description_short']
    if description:
        description = description.replace(' ', '_')[:20]
        return description
    else:
        return internalId

def add_comments(row):
    datatype = row['indicators_dataType']
    target_datatype = erp_mapping.get(datatype, {}).get('datatype')
    if target_datatype == 'CHAR':
        return "Characteristic type [CHAR] is not supported in APM"
    
results = db.select(
    model = V_Transform_Indicators
)

df = pd.DataFrame(results)
df = convert_dataframe(df)

df['propose_ERPCharacteristic'] = df.apply(propose_erp_characteristic, axis=1)
df['input_ERPCharacteristic'] = None
df['propose_APMIndicatorPosition'] = df.apply(propose_apm_indicator_position, axis=1)
df['input_APMIndicatorPosition'] = None
df['propose_APMIndicatorCategory'] = map_indicator_category
df['input_APMIndicatorCategory'] = None
df['comments'] = df.apply(add_comments, axis=1)

columns = CONFIG_GLOBAL.get("udr_columns")
df_export = df[list(columns.keys())]
df_export = df_export.rename(columns=columns)

df_export.to_csv(udr_file, index=False)
log.info(f"{udr_file} generated.")

Once the above user decision report is created, it is sent to the user. The user can validate the proposed fields and can also provide 'inputs'.

| Field Name | Purpose |
| ---------- | ------- |
| Proposed: XXX | Value proposed by the tool based on the proposal rules pre-defined.
| Input: XXX    | User input for the specific field/column

1. The tool analyses and proposes a value for certain fields for migration in the `proposal` column.
2. In case if the user is satisfied with the proposal, they can leave the corresponding `input` column as blank.
3. If the user does not wish to consider the proposal, they can enter a value in the `input` column.
4. Any value entered in the `input` column will be considered as the final value for migration.

**Note:**
1. If the user enters a value in `input` - there is **NO** data validation performed. The input value will be considered during migration.
2. In case if there are any issues in the `input` value, the corresponding TO/indicator migration may fail.
3. User is **NOT** allowed to edit any of the fields other than `input`
4. If the user opens the `.csv` file in `Microsoft Excel`, there may be inconsitencies. Hence the user needs to ensure that all columns of the CSV file are set as `TEXT` columns. This will preserve leading zeros or other formats of the file, if any!

